In [9]:
from dotenv import load_dotenv
import os 
load_dotenv()
import openai 
api_key = os.environ["OPENAI_API_KEY"]

In [10]:
def word_to_vector(word):
    # Define baseic rules for our verctor components
    vector = [0]*9 # initialize a vector of 9 dimension

    # Rule 1 : Length of the word (normalized to a max of characters for simplecity)
    vowels = "aeiou"
    vector[1] = sum(1 for char in word if char in vowels)/len(word)

    # rule 3 : where the word starts with a vowel (1) or not (0)
    vector[2] = 1 if word[0] in vowels else 0

    # Rule 4: Whether the word ends with a vowel (1) or not (0)
    vector[3] = 1 if word[-1] in vowels else 0

    # Rule 5: Percentage of consonants in the word
    vector[4] = sum(1 for char in word if char not in vowels and char.isalpha()) / len(word)

    return vector

# Example usage
word = "nanhi"
vector = word_to_vector(word)
print(f"Word: {word}\n Vector: {vector}")


Word: nanhi
 Vector: [0, 0.4, 0, 1, 0.6, 0, 0, 0, 0]


In [11]:
import numpy as np
def cosine_similarity(vector_a , vector_b):
     
     dot_product = np.dot(vector_a,vector_b)

     norm_a = np.linalg.norm(vector_a)
     norm_b = np.linalg.norm(vector_b)

     similarity = dot_product / (norm_a*norm_b)
     return similarity

word1 = "example"
word2 = "sample"
vector1 = word_to_vector(word1)
vector2 = word_to_vector(word2)

similarity_score = cosine_similarity(vector1,vector2)
print(f"cosine_similarity between '{word1}' and '{word2}' : {similarity_score} ")

cosine_similarity between 'example' and 'sample' : 0.7711403083390078 


In [12]:
import openai
def openAIEmbeddings(input):
    response = openai.embeddings.create(
        input = "input",
        model = "text-embedding-3-small"
    )
    return response.data[0].embedding

print(openAIEmbeddings("Golden Retreiver"))

[0.021148765459656715, 0.02802383527159691, 0.0026298179291188717, -0.047891274094581604, 0.03212958946824074, -0.056488558650016785, 0.004016198683530092, 0.02091454342007637, -0.023394528776407242, -0.006103519815951586, 0.04497040435671806, 0.014879913069307804, -0.021052321419119835, -0.023449640721082687, 0.021327875554561615, -0.017966117709875107, 0.020239436998963356, -0.014032584615051746, 0.0007775787962600589, 0.0454939566552639, 0.05665389075875282, 0.020886989310383797, 0.033397138118743896, -0.005610967054963112, 0.005876187700778246, -0.024978965520858765, -0.011490599252283573, 0.03031093440949917, 0.02649451233446598, -0.03984510153532028, -0.02857494354248047, -0.03879799321293831, 0.01634034886956215, -0.025970958173274994, 0.03893577307462692, -0.006048408802598715, -0.012399927712976933, 0.023160308599472046, -0.024524301290512085, -0.02192031592130661, -0.028657609596848488, -0.021603429690003395, 0.009926830418407917, 0.024317635223269463, -0.013274811208248138, 

In [ ]:
import openai
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores.singlestoredb import SingleStoreDB
import pandas as pd
import requests

# URL of the public .txt file you want to use
file_url = "https://sherlock-holm.es/stories/plain-text/stud.txt"
# file_url = "https://sherlock-holm.es/html/"

# Send a GET request to the file URL
response = requests.get(file_url)

# Proceed if the file was successfully downloaded
if response.status_code == 200:
    file_content = response.text

    # Save the content to a file
    file_path = 'downloaded_example.txt'
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(file_content)

    # Now, you can proceed with your original code using 'downloaded_example.txt'
    # Load and process documents
    loader = TextLoader(file_path)  # Use the downloaded document

    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)

    # Generate embeddings and create a document search database

    from dotenv import load_dotenv
    import os
    load_dotenv()
    api_key = os.environ["OPENAI_API_KEY"]

    embeddings = OpenAIEmbeddings(api_key=api_key)

    # Create Vector Database
    vector_database = SingleStoreDB.from_documents(docs, embeddings, table_name="scarlet")
    # Replace "your_table_name" with your actual table name

    query = "which university did he study?"
    docs = vector_database.similarity_search(query)
    print(docs[0].page_content)

else:
    print("Failed to download the file. Please check the URL and try again.")